In [1]:
get_ipython().magic('env CUDA_VISIBLE_DEVICES =  ')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
import bson 
import tensorflow as tf
import os.path
import time
import logging
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave   # or, whatever image library you prefer
from datetime import datetime

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

env: CUDA_VISIBLE_DEVICES=


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
CATOGORY_HIST_FILE = '/media/rs/FC6CDC6F6CDC25E4/count_hist.csv'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'

In [3]:
catogory_df = pd.read_csv(CATEGORY_NAME_PATH)
print(len(catogory_df['category_level1'].value_counts()))
print(len(catogory_df['category_level2'].value_counts()))
print(len(catogory_df['category_level3'].value_counts()))

49
483
5263


In [4]:
catogory_hist = pd.read_csv(CATOGORY_HIST_FILE)
catogory_hist.columns = ['category_id', 'count']

In [5]:
catogory_hist = catogory_hist.set_index('category_id')
catogory_hist = catogory_hist.reset_index()

In [6]:
# max_num_cout = catogory_hist['count'].max()
# max_num_cout
# catogory_hist['catogory_weight'] = catogory_hist['count'].apply(lambda x : np.log(80348/x+1)).apply(lambda x : x if x<4.else 4.)
# catogory_hist.to_csv(DATASET_PATH + 'catogory_with_weight.csv', index=False)

In [7]:
sample_ratio = dict()

low, high = 0.6, 0.4
range_low, range_high = 50000, 100000
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low 
    
low, high = 1., 0.6
range_low, range_high = 20000, 50000
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low
    
low, high = 1.3, 1.
range_low, range_high = 10000, 20000
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low
    
low, high = 1.8, 1.3
range_low, range_high = 5000, 10000
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low
    
low, high = 4, 1.8
range_low, range_high = 1000, 5000
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low
    
low, high = 6., 4.
range_low, range_high = 500, 1000
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low
    
low, high = 10., 6
range_low, range_high = 100, 500
range_length = range_high - range_low
for num in range(range_low, range_high, 100):
    sample_ratio[num//100] = (num - range_low)*(high - low)/range_length + low
    
sample_ratio[0] = 10.

In [8]:
catogory_hist['sample_count'] = catogory_hist['count'].apply(lambda x : x * sample_ratio[x//100] ).astype(int)
#catogory_hist['sample_count'] = catogory_hist['count'].apply(lambda x : x ).astype(int)

In [9]:
encode_df = catogory_df['category_level1'].value_counts().to_frame().reset_index(drop=False)
encode_df.columns = ['category_level1', 'class_count']

In [10]:
class_count_list = sorted((encode_df['class_count'].values), reverse=True)
catogory_encode_group = [list(range(x)) for x in class_count_list]
catogory_encode_by_group = [code for catogory in catogory_encode_group for code in catogory]
#print(catogory_encode_by_group)
print(class_count_list)

[555, 441, 440, 237, 230, 230, 220, 206, 196, 184, 180, 162, 158, 137, 106, 104, 103, 101, 99, 89, 88, 85, 84, 83, 81, 74, 57, 50, 48, 45, 43, 42, 42, 38, 33, 33, 30, 29, 26, 25, 19, 16, 9, 6, 2, 1, 1, 1, 1]


In [11]:
encode_df['level1_encode'] = list(range(len(encode_df)))
encode_df = encode_df.merge(right = catogory_df[['category_id', 'category_level1']], how='left', on = 'category_level1')
encode_df['catogoty_encode_total'] = list(range(len(encode_df)))
encode_df['catogory_encode_by_group'] = catogory_encode_by_group
encode_df.to_csv(DATASET_PATH + 'hierarchy_encode.csv', index=False)

In [12]:
encode_df

,category_level1,class_count,level1_encode,category_id,catogoty_encode_total,catogory_encode_by_group
0,SPORT,555,0,1000018817,0,0
1,SPORT,555,0,1000019365,1,1
2,SPORT,555,0,1000019369,2,2
3,SPORT,555,0,1000019373,3,3
4,SPORT,555,0,1000019375,4,4
5,SPORT,555,0,1000018949,5,5
6,SPORT,555,0,1000019156,6,6
7,SPORT,555,0,1000019804,7,7
8,SPORT,555,0,1000019041,8,8
9,SPORT,555,0,1000019172,9,9


In [13]:
catogory_df = catogory_df.merge(right = catogory_hist, how = 'left', on='category_id')
catogory_df

,category_id,category_level1,category_level2,category_level3,count,sample_count
0,1000021794,ABONNEMENT / SERVICES,CARTE PREPAYEE,CARTE PREPAYEE MULTIMEDIA,32,32
1,1000012764,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,ABRI FUMEUR,1999,1999
2,1000012776,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,ABRI VELO - ABRI MOTO,44,44
3,1000012768,AMENAGEMENT URBAIN - VOIRIE,AMENAGEMENT URBAIN,FONTAINE A EAU,56,56
4,1000012755,AMENAGEMENT URBAIN - VOIRIE,SIGNALETIQUE,PANNEAU D'INFORMATION EXTERIEUR,341,341
5,1000012738,AMENAGEMENT URBAIN - VOIRIE,SIGNALISATION ROUTIERE,CONE DE SIGNALISATION - PLOT DE SIGNALISATION,72,72
6,1000012732,AMENAGEMENT URBAIN - VOIRIE,SIGNALISATION ROUTIERE,PANNEAU ROUTIER,16,16
7,1000013163,AMENAGEMENT URBAIN - VOIRIE,SIGNALISATION ROUTIERE,RUBAN DE SIGNALISATION,57,57
8,1000012712,AMENAGEMENT URBAIN - VOIRIE,VOIRIE,BLOC PARKING - ARCEAU PARKING,186,186
9,1000012718,AMENAGEMENT URBAIN - VOIRIE,VOIRIE,POTEAU - POTELET,135,135


In [14]:
level1_num_examples_hist = catogory_df[['category_level1', 'sample_count']].groupby(['category_level1'])['sample_count'].agg('sum')
level1_num_examples_hist = level1_num_examples_hist.to_frame()
level1_num_examples_hist = level1_num_examples_hist.reset_index(drop=False)
# level1_num_examples_hist
level1_num_classes_hist = catogory_df['category_level1'].value_counts().to_frame().reset_index()
level1_num_classes_hist.columns = ['category_level1', 'num_classes']

In [15]:
level1_classes_detail = level1_num_classes_hist.merge(right = level1_num_examples_hist, how='left', on='category_level1')

In [16]:
max_num_cout = level1_classes_detail['sample_count'].max()
level1_classes_detail['loss_weight'] = level1_classes_detail['sample_count'].apply(lambda x : np.log(2240814/x+1)).apply(lambda x : x if x<3.else 3.)

In [17]:
level1_classes_detail.to_csv(DATASET_PATH + 'level1_class_detail.csv', index=False)

In [18]:
print([int(x*1000)/1000 for x in list(level1_classes_detail['loss_weight'].values)])

[1.817, 1.528, 1.056, 2.174, 1.622, 2.531, 2.663, 1.103, 1.935, 1.937, 2.898, 1.279, 2.923, 1.095, 2.086, 3.0, 2.218, 3.0, 3.0, 2.123, 3.0, 1.038, 3.0, 3.0, 3.0, 3.0, 1.817, 1.448, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 1.955, 3.0, 1.946, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [19]:
# all_level1_class_hist = dict(zip(list(catogory_df['category_level1'].value_counts().index), list(catogory_df['category_level1'].value_counts().values)))
# all_level1_class_hist